In [2]:
import sys
import argparse
import torch
import logging
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

dataset_path = '/opt/ml/input/data'
train_path = dataset_path + '/train.json'
val_path = dataset_path + '/val.json'
test_path = dataset_path + '/test.json'

def __get_logger():
    """로거 인스턴스 반환
    """

    __logger = logging.getLogger('logger')

    # # 로그 포멧 정의
    formatter = logging.Formatter(fmt="%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    # 스트림 핸들러 정의
    stream_handler = logging.StreamHandler()
    # 각 핸들러에 포멧 지정
    stream_handler.setFormatter(formatter)
    # 로거 인스턴스에 핸들러 삽입
    __logger.addHandler(stream_handler)
    # 로그 레벨 정의
    __logger.setLevel(logging.DEBUG)

    return __logger

logger = __get_logger()


def collate_fn(batch):
    return tuple(zip(*batch))

In [66]:
def test(model, test_loader, device):
    """저장된 model에 대한 prediction 수행

    Args:
        model(nn.Module) : 저장된 모델
        test_loader(Dataloader) : test데이터의 dataloader
        device

    테스트 loader는 배치 사이즈 4 이하이면 작동을 하지 않습니다.   
    """

    size = 256
    transform = A.Compose([A.Resize(256, 256)])
    print('Start prediction.')
    model.eval()

    file_name_list = []
    preds_array = np.empty((0, size*size), dtype=np.long)

    with torch.no_grad():
        for step, (imgs1,imgs2, image_infos) in enumerate(tqdm(test_loader)):

            # inference (512 x 512)
            outs1 = model(torch.stack(imgs1).to(device))
            outs2 = model(torch.stack(imgs2).to(device))
            outs = (outs1 + outs2) / 2
            
            oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()

            # resize (256 x 256)
            temp_mask = []
            for img1,img2, mask in zip(np.stack(imgs1), np.stack(imgs2),oms):
                transformed = transform(image=img1, mask=mask)
                mask = transformed['mask']
                temp_mask.append(mask)

            oms = np.array(temp_mask)

            oms = oms.reshape([oms.shape[0], size*size]).astype(int)
            preds_array = np.vstack((preds_array, oms))

            file_name_list.append([i['file_name'] for i in image_infos])
    print("End prediction.")
    file_names = [y for x in file_name_list for y in x]

    return file_names, preds_array

In [1]:
if torch.cuda.is_available():
    logger.info("*************************************")
    device = torch.device("cuda")
    logger.info(f'There are {torch.cuda.device_count()} GPU(s) available.')
    logger.info(f'We will use the GPU:{torch.cuda.get_device_name(0)}')
    logger.info("*************************************\n")
else:
    logger.info("*************************************")
    logger.info('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
    logger.info("*************************************\n")

In [68]:
import os
import json

import numpy as np
import pandas as pd

from pycocotools.coco import COCO
import cv2

from torch.utils.data import Dataset

dataset_path = '/opt/ml/input/data'

def data_eda():
    """데이터 eda 및 학습 데이터에 필요한 데이터 프레임 가져오기 

    Returns:
        sorted_df(DataFrame) : 로드 시 필요정보를 가지는 데이터프레임 반환
    """

    anns_file_path = dataset_path + '/' + 'train.json'
    
    # Read annotations
    with open(anns_file_path, 'r') as f:
        dataset = json.loads(f.read())
        
    categories = dataset['categories']
    anns = dataset['annotations']
    imgs = dataset['images']
    nr_cats = len(categories)
    nr_annotations = len(anns)
    nr_images = len(imgs)
    
    # Load categories and super categories
    cat_names = []
    super_cat_names = []
    super_cat_ids = {}
    super_cat_last_name = ''
    nr_super_cats = 0
    
    for cat_it in categories:
        cat_names.append(cat_it['name'])
        super_cat_name = cat_it['supercategory']
        
        # Adding new supercat
        if super_cat_name != super_cat_last_name:
            super_cat_names.append(super_cat_name)
            super_cat_ids[super_cat_name] = nr_super_cats
            super_cat_last_name = super_cat_name
            nr_super_cats += 1
            
    cat_histogram = np.zeros(nr_cats,dtype=int)
    for ann in anns:
        cat_histogram[ann['category_id']] += 1
        
    # # Initialize the matplotlib figure
    # f, ax = plt.subplots(figsize=(5,5))
    
    # Convert to DataFrame
    df = pd.DataFrame({'Categories': cat_names, 'Number of annotations': cat_histogram})
    df = df.sort_values('Number of annotations', 0, False)

    # category labeling 
    sorted_temp_df = df.sort_index()
    
    # background = 0 에 해당되는 label 추가 후 기존들을 모두 label + 1 로 설정
    sorted_df = pd.DataFrame(["Backgroud"], columns = ["Categories"])
    sorted_df = sorted_df.append(sorted_temp_df, ignore_index=True)

    return sorted_df

def get_classname(classID, cats):
    for i in range(len(cats)):
        if cats[i]['id']==classID:
            return cats[i]['name']
    return "None"

class CustomDataLoader(Dataset):
    """ dataloader의 정의
    """
    def __init__(self, data_dir, mode = 'train', transform1 = None, transform2 = None):
        super().__init__()
        self.mode = mode
        self.transform1 = transform1
        self.transform2 = transform2
        self.coco = COCO(data_dir)
        sorted_df = data_eda()
        self.category_names = list(sorted_df.Categories)
        
    def __getitem__(self, index: int):
        # dataset이 index되어 list처럼 동작
        image_id = self.coco.getImgIds(imgIds=index)
        image_infos = self.coco.loadImgs(image_id)[0]
        
        # cv2 를 활용하여 image 불러오기
        paths = os.path.join(dataset_path, image_infos['file_name'])
        images = cv2.imread(os.path.join(dataset_path, image_infos['file_name']))
        # images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.float32)
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.uint8)
        # images /= 255.0
        
        if (self.mode in ('train', 'val')):
            ann_ids = self.coco.getAnnIds(imgIds=image_infos['id'])
            anns = self.coco.loadAnns(ann_ids)

            # Load the categories in a variable
            cat_ids = self.coco.getCatIds()
            cats = self.coco.loadCats(cat_ids)

            # masks : size가 (height x width)인 2D
            # 각각의 pixel 값에는 "category id + 1" 할당
            # Background = 0
            masks = np.zeros((image_infos["height"], image_infos["width"]))
            # Unknown = 1, General trash = 2, ... , Cigarette = 11
            for i in range(len(anns)):
                className = get_classname(anns[i]['category_id'], cats)
                pixel_value = self.category_names.index(className)
                masks = np.maximum(self.coco.annToMask(anns[i])*pixel_value, masks)
            # masks = masks.astype(np.float32)
            masks = masks.astype(np.uint8)

            # transform -> albumentations 라이브러리 활용
            if self.transform1 is not None:
                transformed = self.transform(image=images, mask=masks)
                images = transformed["image"]
                masks = transformed["mask"]
            
            return paths, images, masks, image_infos
        
        if self.mode == 'test':
            # transform -> albumentations 라이브러리 활용
            if self.transform1 is not None:
                transformed1 = self.transform1(image=images)
                images1 = transformed1["image"]
            if self.transform2 is not None:
                transformed2 = self.transform2(image=images)
                images2 = transformed2["image"]
            
            return images1,images2, image_infos
    
    
    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())

In [2]:
# model 갈아 끼우기
model_path = '/opt/ml/saved/check.pt' 

model = smp.DeepLabV3Plus(
        encoder_name="resnext50_32x4d",
        encoder_weights= "imagenet",
        in_channels=3,
        classes=12
        )
# model = ResNextDeepLabV3AllTrain()
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint)
model.to(device)

test_transform1 = A.Compose([
    A.Normalize(
                 mean=(0.46446795,0.44277694,0.42146815), 
                 std=(0.21137457,0.20805008,0.21553354), 
                 max_pixel_value=255.0, 
                 p=1.0),
    ToTensorV2()
])

test_transform2 = A.Compose([
    A.CLAHE(p = 1.0),
    A.Normalize(
                 mean=(0.46446795,0.44277694,0.42146815), 
                 std=(0.21137457,0.20805008,0.21553354), 
                 max_pixel_value=255.0, 
                 p=1.0),
    ToTensorV2()
])

# test dataset
test_dataset = CustomDataLoader(data_dir=test_path, 
                                mode='test', 
                                transform1=test_transform1,
                                transform2=test_transform2
                               )

# test dataloader
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
batch_size= 8,
num_workers=2,
collate_fn=collate_fn
)

# sample_submisson.csv 열기
submission = pd.read_csv('/opt/ml/submission/sample_submission.csv', index_col=None)

# test set에 대한 prediction
file_names, preds = test(model, test_loader, device)

# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, ignore_index=True)

# submission.csv로 저장
submission.to_csv("/opt/ml/submission/" + "check" + ".csv", index=False)

In [70]:
x = torch.randn([2, 3, 512, 512])
y = torch.randn([2, 3, 512, 512])
z = x + y

In [3]:
z.shape